In [1]:
!pip install neuralforecast

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.6/170.6 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.9/776.9 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 MB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.4/413.4 kB 33.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 806.1/806.1 kB 30.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 230.6/230.6 kB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 4.9 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import numpy as np
import torch
from typing import Optional, Union, Tuple


import torch.nn.functional as F
from torch.distributions import Distribution
from torch.distributions import Bernoulli, Normal, StudentT, Poisson, NegativeBinomial

from torch.distributions import constraints


from neuralforecast import NeuralForecast
from neuralforecast.models import TFT
from neuralforecast.tsdataset import TimeSeriesDataset
from neuralforecast.utils import AirPassengers, AirPassengersPanel, AirPassengersStatic
from neuralforecast.losses.numpy import rmse, mape
from pandas.tseries.holiday import USFederalHolidayCalendar
from pandas.tseries.offsets import CustomBusinessDay
from neuralforecast.losses.pytorch import MSE, MAE, MQLoss, DistributionLoss

In [3]:
from numpy.random import seed
from random import randrange

In [4]:
df = pd.read_excel("SP100_vol (clean).xlsx", index_col="Date",parse_dates=True)
columns=df.columns
df_static=pd.read_excel("Static_df.xlsx")
n_series=len(df.columns)

In [5]:
n_inputs = [3,5,10,15,21,42,84]
hidden_size = [50,100,150,200,300,500]
epochs=[50,100,150,250,350,450,550,650,750]
dropout=[0,0.2,0.3,0.4]
n_head=[2,4,5,8,10,20]
attn_dropout=[0,0.2,0.3,0.4]
learning_rate=[0.0005,0.0001,0.00005,0.00001]
losses=[MSE(),MAE(),MQLoss(level=[80, 90]),DistributionLoss(distribution='StudentT', level=[80, 90])]
num_lr_decays=[5,3,2,1]
scaler_type=["robust","standard",'minmax']

# Test

In [6]:
test_length=int(len(df['NKE'])*0.3)
df1=df.unstack().reset_index(name="Value").rename(columns={"level_0":"unique_id", "Date":"ds", "Value":"y"})

In [7]:
RMSE = [0.004920286914366591, 0.004831135823145892, 0.004857330148582938, 0.004878033614934677]
MAE = [0.003098433599799367, 0.002980252542294194, 0.003025359498354602, 0.003026809727156642]
QLIKE =[0.04432469562112369, 0.04387984074214025, 0.04412221012948656, 0.043958573968901925]
MAPE =[0.2425595722511946, 0.2280365447654647, 0.2338368911205584, 0.2336714571061285]
H=5

for m in range (16):
  randomseed=randrange(129228148)
  model=TFT(h=H, input_size=15, stat_exog_list=['AMT', 'COP', 'CVX', 'AXP', 'BAC', 'C', 'CB', 'CME', 'GS', 'JPM', 'MS',
         'PNC', 'USB', 'WFC', 'BA', 'CAT', 'GE', 'HON', 'LMT', 'MMM', 'UNP',
         'UPS', 'AAPL', 'ACN', 'ADBE', 'ADP', 'CMCSA', 'CRM', 'CSCO', 'DIS',
         'FIS', 'FISV', 'GOOG', 'IBM', 'INTC', 'INTU', 'MA', 'MSFT', 'NFLX',
         'NVDA', 'ORCL', 'QCOM', 'T', 'TXN', 'VZ', 'COST', 'KO', 'MO', 'PEP',
         'PG', 'WMT', 'D', 'DUK', 'SO', 'ABT', 'AMGN', 'BDX', 'BMY', 'BSX', 'CI',
         'CVS', 'DHR', 'GILD', 'ISRG', 'JNJ', 'LLY', 'MDT', 'MRK', 'PFE', 'SYK',
         'TMO', 'UNH', 'AMZN', 'HD', 'LOW', 'MCD', 'NKE', 'SBUX', 'TGT', 'TJX'],
         hidden_size=500,
         n_head=20, attn_dropout=0.4, dropout=0.2, loss=MSE(),
         max_steps=350, learning_rate=learning_rate[3],
         num_lr_decays=num_lr_decays[3],
         step_size=1,
         scaler_type='minmax',
                random_seed=randomseed)
  fcst = NeuralForecast(models=[model],freq=CustomBusinessDay(calendar=USFederalHolidayCalendar()))
  forecasts = fcst.cross_validation(df=df1,val_size=H,static_df=df_static,n_windows=None, test_size=test_length-test_length%H,step_size=H)
  forecasts = forecasts.dropna()
  Y_hat=forecasts["TFT"].values
  Y_true=forecasts["y"].values
  RMSE.append(np.sqrt(np.sum(((Y_true-Y_hat)**2))/len(Y_true)))
  QLIKE.append((np.sum(Y_true/Y_hat-np.log(abs(Y_true)/abs(Y_hat))-1)/len(Y_true)))
  MAE.append(np.sum(abs(Y_true-Y_hat))/len(Y_true))
  MAPE.append(np.sum(abs(Y_true-Y_hat)/(Y_true))/len(Y_true))

print(f"RMSE mean: {np.mean(RMSE)} | QLIKE mean: {np.mean(QLIKE)} | MAE mean: {np.mean(MAE)} | MAPE mean: {np.mean(MAPE)}")
print(f"RMSE std: {np.std(RMSE)} | QLIKE std: {np.std(QLIKE)} | MAE std: {np.std(MAE)} | MAPE std: {np.std(MAPE)}")
from scipy.stats import anderson
print(f"RMSE: {anderson(RMSE)}")
print(f"QLIKE: {anderson(QLIKE)}")
print(f"MAE: {anderson(MAE)}")
print(f"MAPE: {anderson(MAPE)}")

df_errors = pd.DataFrame({
    'RMSE': RMSE,
    'QLIKE': QLIKE,
    'MAE': MAE,
    'MAPE': MAPE
})

df_errors.to_excel("TFT forecasts.xlsx")


INFO:lightning_fabric.utilities.seed:Seed set to 97719849


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

RMSE mean: 0.004846938359260955 | QLIKE mean: 0.04469166899552961 | MAE mean: 0.003053214725648407 | MAPE mean: 0.2387235410755313
RMSE std: 2.8522525778848413e-05 | QLIKE std: 0.000946352950918368 | MAE std: 6.687313310260866e-05 | MAPE std: 0.009837749182848363
RMSE: AndersonResult(statistic=0.32097208410430866, critical_values=array([0.506, 0.577, 0.692, 0.807, 0.96 ]), significance_level=array([15. , 10. ,  5. ,  2.5,  1. ]), fit_result=  params: FitParams(loc=0.004846938359260955, scale=2.9263493995889223e-05)
 success: True
 message: '`anderson` successfully fit the distribution to the data.')
QLIKE: AndersonResult(statistic=2.1189670273508874, critical_values=array([0.506, 0.577, 0.692, 0.807, 0.96 ]), significance_level=array([15. , 10. ,  5. ,  2.5,  1. ]), fit_result=  params: FitParams(loc=0.04469166899552961, scale=0.00097093764107415)
 success: True
 message: '`anderson` successfully fit the distribution to the data.')
MAE: AndersonResult(statistic=0.5209294362346952, crit